In [8]:
#Esto me sirve para el correcto funcionamiento de las funciones importadas en este notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Preparando los datos

## Leemos los archivos

Iniciamos seleccionando cuatro conjuntos de datos obtenidos [**NBA Stats (1947-present)**](https://www.kaggle.com/datasets/sumitrodatta/nba-aba-baa-stats) que contienen información sobre jugadores a lo largo de todas las temporadas de la NBA. La elección de estos cuatro conjuntos de datos se fundamenta en las siguientes razones:

*   **df_premios_jugadores**: Este conjunto de datos presenta información detallada sobre todos los premios otorgados a lo largo de la historia de la NBA, incluyendo la proporción de votos obtenidos por los jugadores que fueron considerados para recibir dichos galardones. En última instancia, este dataframe se utilizará para el análisis del **premio MVP**.

*   **estadisticas_avanzadas**: Contiene una amplia variedad de estadísticas que son fundamentales para evaluar el rendimiento de los jugadores a lo largo de una temporada. Estas estadísticas incluyen índices de eficacia, el **PER** (Player Efficiency Rating), el impacto del rendimiento en la cancha y otros indicadores relevantes.

*   **estadisticas_tiro**: Al igual que el conjunto de datos de **estadisticas_avanzadas**, este conjunto proporciona diversas estadísticas sobre los jugadores en distintas temporadas. Sin embargo, se enfoca específicamente en las estadísticas relacionadas con el tiro, incluyendo mediciones de precisión desde diversas distancias y la proporción de cada tipo de tiro.

*   **estadisticas_por_juego**: Este conjunto de datos proporciona información detallada, partido tras partido, sobre las estadísticas individuales de un jugador mientras está en la cancha. Estos datos son especialmente valiosos, ya que permiten evaluar el rendimiento de los jugadores en cada juego, lo que es fundamental para comprender su contribución en cada encuentro.

In [2]:
from utils.paths import crear_funcion_directorio
import pandas as pd

data_dir = crear_funcion_directorio("data") #Me hago el acceso directo al directorio "data"

df_premios_jugadores = pd.read_csv(data_dir("raw", "Player Award Shares.csv"))
estadisticas_avanzadas = pd.read_csv(data_dir("raw", "Advanced.csv"))
estadisticas_tiro = pd.read_csv(data_dir("raw", "Player Shooting.csv"))
estadisticas_por_juego = pd.read_csv(data_dir("raw", "Player Per Game.csv"))

## Construyendo el conjunto de datos

Empezamos por dar un vistazo a las columnas de los datasets para ver cuales tienen en común entre sí para luego juntarlos en un **único dataset** el cual traerá toda la información a utilizar en el proyecto:

In [7]:
from utils.workit import imprimir_columnas

imprimir_columnas(df_premios_jugadores)
imprimir_columnas(estadisticas_avanzadas)
imprimir_columnas(estadisticas_tiro)
imprimir_columnas(estadisticas_por_juego)

df:
['season', 'award', 'player', 'age', 'tm', 'first', 'pts_won', 'pts_max', 'share', 'winner', 'seas_id', 'player_id']

df:
['seas_id', 'season', 'player_id', 'player', 'birth_year', 'pos', 'age', 'experience', 'lg', 'tm', 'g', 'mp', 'per', 'ts_percent', 'x3p_ar', 'f_tr', 'orb_percent', 'drb_percent', 'trb_percent', 'ast_percent', 'stl_percent', 'blk_percent', 'tov_percent', 'usg_percent', 'ows', 'dws', 'ws', 'ws_48', 'obpm', 'dbpm', 'bpm', 'vorp']

df:
['seas_id', 'season', 'player_id', 'player', 'birth_year', 'pos', 'age', 'experience', 'lg', 'tm', 'g', 'mp', 'fg_percent', 'avg_dist_fga', 'percent_fga_from_x2p_range', 'percent_fga_from_x0_3_range', 'percent_fga_from_x3_10_range', 'percent_fga_from_x10_16_range', 'percent_fga_from_x16_3p_range', 'percent_fga_from_x3p_range', 'fg_percent_from_x2p_range', 'fg_percent_from_x0_3_range', 'fg_percent_from_x3_10_range', 'fg_percent_from_x10_16_range', 'fg_percent_from_x16_3p_range', 'fg_percent_from_x3p_range', 'percent_assisted_x2p_fg', '

Podemos observar que las columnas **'seas_id', 'season', 'player_id' y 'player'** funcionan como **identificadores** que facilitan la combinación de los
conjuntos de datos, si es necesario. Por otro lado, hay columnas comunes entre **estadísticas avanzadas** y **estadísticas de tiro** que contienen información idéntica. En consecuencia, se planea eliminar estas columnas de uno de los dos conjuntos de datos para evitar duplicaciones al combinarlos:

In [ ]:
#Guardo en una variable las columnas que me sirven para juntar los datasets
identificadores = ['seas_id', 'season', 'player_id','player']

#Me quedo con las columnas que me serán útiles de las estadisticas de tiro
columnas_utiles = [columna for columna in list(estadisticas_tiro.columns) if columna not in list(estadisticas_avanzadas.columns)]

#Como en las lineas anteriores eliminamos las columnas en común también se eliminaron los identificadores,
#por lo tanto los vuelvo a incluir en la lista de columnas útiles
for columna in identificadores:
  columnas_utiles.append(columna)

#Ahora me quedo con un dataframe con puramente información nueva
estadisticas_tiro = estadisticas_tiro.drop([columna for columna in list(estadisticas_tiro.columns) if columna not in columnas_utiles], axis = 1)

Aplicamos el mismo procedimiento para **estadisticas por juego**:

In [ ]:
#Me quedo con las columnas que me serán útiles de las estadisticas por partido
columnas_utiles = [columna for columna in list(estadisticas_por_juego.columns) if columna not in list(estadisticas_avanzadas.columns) and columna not in list(estadisticas_tiro.columns)]

#Vuelvo a agregarle los identificadores
for columna in identificadores:
  columnas_utiles.append(columna)

#Ahora me quedo con un dataframe con puramente información nueva
estadisticas_por_juego = estadisticas_por_juego.drop([columna for columna in list(estadisticas_por_juego.columns) if columna not in columnas_utiles], axis = 1)

En el caso de **df_premios_jugadores**, como se dijo anteriormente, este proyecto tratará sobre el **premio MVP** y de este conjunto de datos la variable que nos va a servir para nuestro objetivo será la columna **'mvp_share'**, que trae información sobre la proporción de votos que recibió un jugador para el premio. Por lo tanto procedo a quedarme con esa columna:

In [ ]:
#Me quedo con las instancias que traen información solo del premio al MVP
df_premios_jugadores = df_premios_jugadores[df_premios_jugadores["award"]=="nba mvp"]

#Eliminamos las columnas que no usaremos
df_premios_jugadores = df_premios_jugadores.drop(['award','age','tm','first','pts_won','pts_max','winner'], axis = 1)
df_premios_jugadores.rename(columns={'share': 'mvp_share'}, inplace=True)

Tras realizar las modificaciones necesarias, procedemos a fusionar los cuatro conjuntos de datos en uno único, denominado **df_data_jugadores**. Este conjunto de datos principal será el pilar fundamental de nuestro proyecto.

Además, como se puede observar, hemos aplicado un filtro para restringir el análisis a las temporadas **más recientes**. Esta selección tiene como objetivo mantener la coherencia con los cambios que han tenido lugar en la liga, evitando así posibles sesgos en nuestros modelos de trabajo.

In [ ]:
#Trabajo con las temporadas entre 2000 y 2023
estadisticas_tiro = estadisticas_tiro.loc[(estadisticas_tiro["season"] >=2000)]
estadisticas_avanzadas = estadisticas_avanzadas.loc[(estadisticas_avanzadas["season"] >=2000)]
df_premios_jugadores = df_premios_jugadores.loc[(df_premios_jugadores["season"] >=2000)]
estadisticas_por_juego = estadisticas_por_juego.loc[(estadisticas_por_juego["season"] >=2000)]

#Junto los datasets en uno solo que será el usado para el proyecto
df_data_jugadores = estadisticas_avanzadas.merge(estadisticas_tiro, on= identificadores, how='inner')
df_data_jugadores = df_data_jugadores.merge(df_premios_jugadores, on= identificadores, how='left')
df_data_jugadores = df_data_jugadores.merge(estadisticas_por_juego, on= identificadores, how='left')

#Finalmente miramos el dataset resultante
df_data_jugadores.info()

Del output anterior pudimos notar que en la columna **'birth_year'** hay una gran cantidad de valores faltantes, además que saber la fecha en que nació un jugador no será de gran relevancia para los propositos del análisis, por lo tanto procedemos a eliminarla:

In [ ]:
df_data_jugadores.drop(['birth_year'], axis = 1, inplace= True)

Del mismo modo, existen columnas, como **'experience'**, que resultan redundantes en el análisis, ya que reflejan la cantidad de años que un jugador ha estado en la liga, información que puede deducirse fácilmente a partir de la columna **'season'**. Por lo tanto, procedemos a eliminarla:

In [ ]:
df_data_jugadores.drop(['experience'], axis = 1, inplace= True)

La columna **'tm'** contiene información sobre los equipos, y en ella encontramos una entrada con el nombre **'TOT'**. Sin embargo, **'TOT'** no representa un equipo en el sentido tradicional; más bien, se refiere a estadísticas totales de jugadores que **cambiaron de equipo** a mitad de temporada. Esta situación podría causar problemas, ya que se traduciría en datos duplicados para diferentes jugadores. Por lo tanto, hemos decidido eliminar estas entradas para garantizar la integridad de nuestros datos:

In [ ]:
df_data_jugadores.drop(df_data_jugadores[df_data_jugadores['tm'] == 'TOT'].index, inplace=True)

Ahora que disponemos del conjunto de datos que alberga la información relevante, nos encontramos en la fase final, que implica la selección de los jugadores idóneos para llevar a cabo un análisis preciso.

En un primer acercamiento, notamos que en la columna **'mvp_share'** (nuestro objetivo) contamos con instancias que representan temporadas en las que ciertos jugadores obtuvieron votos para el premio al **Jugador Más Valioso**. Son estos jugadores en quienes basaremos nuestro análisis, con el propósito de identificar acontecimientos significativos en el transcurso de sus carreras antes y después de recibir votos o ganar el premio. Incluso en casos excepcionales en los que algunos jugadores han ganado el premio en **múltiples ocasiones**, nuestro objetivo es identificar patrones comunes. Esto nos permitirá obtener una comprensión clara de los criterios que influyen en la obtención de tan prestigioso galardón.

In [ ]:
#Filtro las filas donde 'mvp_share' no sea un valor faltante
df_data_jugadores_mvp = df_data_jugadores[df_data_jugadores['mvp_share'].notna()]

#Armo una lista de nombres de jugadores con valor en 'mvp_share'
nombres_con_mvp_share = df_data_jugadores_mvp['player'].unique()

#Me quedo con las instancias cuyo nombre está en la lista
df_data_jugadores = df_data_jugadores[df_data_jugadores['player'].isin(nombres_con_mvp_share)]

#Veo el resultado
df_data_jugadores.info()

Como mencionamos anteriormente, para hacer un análisis más robusto nos enfocaremos en temporadas cercanas a la temporada en la cual un jugador recibió votos, entonces dicho esto procedemos a hacer un último filtrado:

In [ ]:
def crear_df_jugador(jugador:str, dataframe):
    df_info_jugador = dataframe[dataframe['player'] == jugador]

    return df_info_jugador

def filtrar_temporadas_jugador(df_jugador):

    #Inicializo una lista para quedarme con las temporadas de interés
    temporadas_interesantes = []

    #Con el dataframe del jugador voy iterando fila por fila
    for index, fila in df_jugador.iterrows():

      #Voy iterando hata encontrame con una fila sin valor nulo en 'mvp_share'
        if not pd.isna(fila['mvp_share']):

            #Usando el criterio me quedo con las temporadas de interés
            temporada_actual = fila['season']
            temporada_anterior = temporada_actual - 1
            temporada_siguiente = temporada_actual + 1

            #Incluyo las temporadas a la lista
            temporadas_interesantes.append(temporada_actual)

            if temporada_anterior >= 2000:
              temporadas_interesantes.append(temporada_anterior)

            if temporada_siguiente <= 2023:
              temporadas_interesantes.append(temporada_siguiente)

    #Aplico el filtro al DataFrame del jugador para incluir solo las temporadas de interés
    filtro_temporadas_interesantes = df_jugador['season'].isin(temporadas_interesantes)
    jugador_filtrado = df_jugador[filtro_temporadas_interesantes]

    return jugador_filtrado


def filtrar_temporadas_jugadores(dataframe):
    jugadores_unicos = dataframe['player'].unique()  # Obtiene la lista de jugadores únicos

    # Inicializa un DataFrame vacío donde se concatenarán los resultados
    resultado_final = pd.DataFrame()

    # Itera sobre la lista de jugadores únicos
    for jugador in jugadores_unicos:
        jugador_filtrado = filtrar_temporadas_jugador(crear_df_jugador(jugador, dataframe))
        resultado_final = pd.concat([resultado_final, jugador_filtrado])

    return resultado_final

#Aplico el filtro a todos los jugadores en el DataFrame general
df_data_jugadores = filtrar_temporadas_jugadores(df_data_jugadores)

Después de aplicar el filtro necesario, hemos obtenido el conjunto de datos que emplearemos en el proyecto. Como paso final, procederemos a **imputar el valor 0** en las celdas con datos faltantes en la variable **'mvp_share'**. Esta acción se justifica, ya que la ausencia de un valor en esta columna indica que el jugador no recibió votos, lo que es equivalente a asignarle un valor de 0 votos.

In [ ]:
#Imputo los ceros
df_data_jugadores['mvp_share'].fillna(0, inplace=True)

#Miro los resultados
df_data_jugadores.info()

## Separamos los datos

Después de adquirir el conjunto de datos con el que trabajaremos, debemos realizar un último paso: la división de los datos en conjuntos de **entrenamiento y prueba**:


In [ ]:
#Selecciono a los jugadores
df_giannis = crear_df_jugador('Giannis Antetokounmpo',df_data_jugadores)
df_embiid = crear_df_jugador('Joel Embiid',df_data_jugadores)
df_jokic = crear_df_jugador('Nikola Jokić',df_data_jugadores)
df_harden = crear_df_jugador('James Harden',df_data_jugadores)
df_lebron = crear_df_jugador('LeBron James',df_data_jugadores)

#Selecciono ahora sus nombres
nombres_a_eliminar = ['Giannis Antetokounmpo', 'Joel Embiid','Nikola Jokić','LeBron James','James Harden']

#Los elimino del dataset original para ahora convertirlo en un train
df_data_jugadores_train  = df_data_jugadores[~df_data_jugadores['player'].isin(nombres_a_eliminar)]
df_data_jugadores_train  = df_data_jugadores_train[df_data_jugadores_train['season'] < 2022]

#Junto todos esos dataframes en el test que evaluaremos al final
df_data_jugadores_test  = df_data_jugadores[df_data_jugadores['season'] >= 2022]

Guardo los dataframes:

In [ ]:
# Guardar df_giannis
df_giannis.to_csv(data_dir("processed", "df_giannis.csv"), index=False)

# Guardar df_embiid
df_embiid.to_csv(data_dir("processed", "df_embiid.csv"), index=False)

# Guardar df_jokic
df_jokic.to_csv(data_dir("processed", "df_jokic.csv"), index=False)

# Guardar df_harden
df_harden.to_csv(data_dir("processed", "df_harden.csv"), index=False)

# Guardar df_lebron
df_lebron.to_csv(data_dir("processed", "df_lebron.csv"), index=False)

# Guardar df_data_jugadores_train
df_data_jugadores_train.to_csv(data_dir("processed", "df_data_jugadores_train.csv"), index=False)

# Guardar df_data_jugadores_test
df_data_jugadores_test.to_csv(data_dir("processed", "df_data_jugadores_test.csv"), index=False)

# Guardar df_data_jugadores
df_data_jugadores.to_csv(data_dir("processed", "df_data_jugadores.csv"), index=False)

Como se ha podido observar, consideramos como instancias de entrenamiento a las que se ubicaban entre las temporadas **2000 y 2021**, y como datos de prueba dejamos fuera **las últimas dos temporadas**, esto con el objetivo de predecir las premiaciones más recientes.

Además decidimos realizar una separación un tanto particular en nuestro enfoque. Optamos por seleccionar **ciertos jugadores** que han ganado el premio en los años recientes y **apartarlos del conjunto de datos de entrenamiento**. Esta elección se basa en la idea de utilizarlos como un grupo de **"prueba"** para evaluar adecuadamente la capacidad de generalización de nuestro modelo.